In [1]:

import torch
from sklearn.model_selection import train_test_split
from Code.Models.hybrid_recsys import *
from Code.Models.nuemf_hybrid import *
from Code.Models.content_based import ContentBasedRecommender
from Code.Models.collaborative import CollaborativeFilteringRecommender
from Code.utils.evaluation import * 
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import random

In [ ]:
# Set device to MPS if available.
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

# Load your data (adjust file paths as needed)
movies_df = pd.read_csv("Data/movies.csv", engine='python')
ratings_df = pd.read_csv("Data/ratings.csv", engine='python')

# Preprocess movies: (Assume you already created genre_features and mapping below)
movies_df['genre_list'] = movies_df['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])
unique_genres = sorted(list({genre for genres in movies_df['genre_list'] for genre in genres}))
for genre in unique_genres:
    movies_df[genre] = movies_df['genre_list'].apply(lambda x: int(genre in x))
genre_features = movies_df[unique_genres].values


train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)

# Build a pivot table and compute user means.
ratings_pivot = train_df.pivot_table(index='userId', columns='movieId', values='rating')
user_means = ratings_pivot.mean(axis=1)

# Create mapping dictionaries.
user_ids_all = ratings_df['userId'].unique()
movie_ids_all = ratings_df['movieId'].unique()
user2index = {u: i for i, u in enumerate(sorted(user_ids_all))}
movie2index = {m: i for i, m in enumerate(sorted(movie_ids_all))}

# Create a dataset for your PyTorch models.
class MovieLensDataset(torch.utils.data.Dataset):
    def __init__(self, ratings_df, movies_df, user2index, movie2index, genre_features, user_means):
        self.df = ratings_df.copy()
        self.user2index = user2index
        self.movie2index = movie2index
        self.df['user_idx'] = self.df['userId'].apply(lambda x: self.user2index[x])
        self.df['movie_idx'] = self.df['movieId'].apply(lambda x: self.movie2index[x])
        self.df['rating_norm'] = self.df.apply(lambda row: row['rating'] - user_means.loc[row['userId']], axis=1)
        movieid_to_index = {mid: idx for idx, mid in enumerate(movies_df['movieId'].values)}
        self.df['genre_vector'] = self.df['movieId'].apply(lambda x: genre_features[movieid_to_index[x]])
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return (torch.tensor(row['user_idx'], dtype=torch.long),
                torch.tensor(row['movie_idx'], dtype=torch.long),
                torch.tensor(row['rating_norm'], dtype=torch.float),
                torch.tensor(row['genre_vector'], dtype=torch.float))

hybrid_train_dataset = MovieLensDataset(train_df, movies_df, user2index, movie2index, genre_features, user_means)
train_loader = DataLoader(hybrid_train_dataset, batch_size=64, shuffle=True)


hybrid_test_dataset = MovieLensDataset(test_df, movies_df, user2index, movie2index, genre_features, user_means)
test_loader = DataLoader(hybrid_test_dataset, batch_size=64, shuffle=True)

# Build dictionary for positive interactions (rating >= 4.0).
def build_positive_user_items(ratings_df, user2index, movie2index, threshold=4.0):
    pos_items = {}
    for _, row in ratings_df.iterrows():
        if row['rating'] >= threshold:
            u = user2index[row['userId']]
            m = movie2index[row['movieId']]
            pos_items.setdefault(u, set()).add(m)
    return pos_items

positive_user_items = build_positive_user_items(train_df, user2index, movie2index, threshold=4.0)
all_movie_indices = list(movie2index.values())
# Inverse mapping from movie index to original movie ID.
inv_movie2index = {v: k for k, v in movie2index.items()}
# Mapping from movieId (original) to row index in movies_df.
movieid_to_index = {mid: idx for idx, mid in enumerate(movies_df['movieId'].values)}




Using device: mps


### RANDOM: Baseline

In [3]:


def random_recommender(user_id):
    """
    Randomly recommends 10 movies that the user has not already rated in the training set.
    
    Args:
        user_id: The original user ID.
    
    Returns:
        A list of 10 movie IDs selected at random.
    """
    # Get the set of movies that the user rated in the training data.
    rated = set(train_df[train_df['userId'] == user_id]['movieId'])
    
    # Candidate movies are those not rated by the user.
    candidate_movies = [movie for movie in movie2index.keys() if movie not in rated]
    
    # If there are fewer than k candidates, return them all.
    if len(candidate_movies) < 10:
        return candidate_movies
    
    # Randomly sample 10 movies from the candidates.
    return random.sample(candidate_movies, 10)

# Evaluate the random recommender baseline.
print("\nEvaluating Random Recommender Baseline:")
evaluate_recommender_metrics(random_recommender, 
                     user_ids=test_df['userId'].unique(), 
                     test_df=test_df, 
                     k=10, 
                     threshold=4.0)



Evaluating Random Recommender Baseline:
Avg Precision@10: 0.0012
Avg Recall@10: 0.0013
Avg NDCG@10: 0.0019


(0.0012170385395537525, 0.001305731566045504, 0.0018739750936774763)

## Content Based and Collaborative Recommender

In [4]:
# Initialize and fit the content-based recommender.
cb_recommender = ContentBasedRecommender(movies_df, genre_features)
cb_recommender.fit(train_df, threshold=4.0)

# Initialize and fit the collaborative filtering recommender.
cf_recommender = CollaborativeFilteringRecommender()
cf_recommender.fit(train_df)

# Wrap the recommend methods to match the evaluation function interface.
def content_based_wrapper(user_id):
    return cb_recommender.recommend(user_id, k=10)

def collaborative_wrapper(user_id):
    return cf_recommender.recommend(user_id, k=10)

print("Evaluating Content-Based Recommender:")
evaluate_recommender_metrics(content_based_wrapper, user_ids=test_df['userId'].unique(), test_df=test_df, k=10, threshold=4.0)

print("\nEvaluating Collaborative Filtering Recommender:")
evaluate_recommender_metrics(collaborative_wrapper, user_ids=test_df['userId'].unique(), test_df=test_df, k=10, threshold=4.0)


Evaluating Content-Based Recommender:
Avg Precision@10: 0.0088
Avg Recall@10: 0.0080
Avg NDCG@10: 0.0106

Evaluating Collaborative Filtering Recommender:
Avg Precision@10: 0.1645
Avg Recall@10: 0.1580
Avg NDCG@10: 0.2210


(0.16448979591836738, 0.15795731340964334, 0.22102827093694025)

### LIGHTFM Recommender (Library model)

In [5]:
from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

/opt/anaconda3/envs/MLL/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
# Normalize ratings for LightFM (shift normalized ratings to be nonnegative)
ratings_pivot = train_df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
shift = abs(ratings_pivot.min().min())
normalized_ratings = ratings_pivot.copy() + shift

# Create and fit the LightFM dataset using training data only.
lfm_dataset = LFMDataset()
lfm_dataset.fit(
    users=train_df['userId'].unique(),
    items=train_df['movieId'].unique(),
    item_features=unique_genres
)

# Build interaction data 
def build_interactions(df, normalized_ratings):
    interactions = []
    for _, row in df.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        weight = (
            normalized_ratings.loc[user_id, movie_id] 
            if (movie_id in normalized_ratings.columns) 
            else 0
        )
        interactions.append((user_id, movie_id, weight))
    return interactions

train_interactions_data = build_interactions(train_df, normalized_ratings)
test_interactions_data = build_interactions(test_df, normalized_ratings)

train_interactions, _ = lfm_dataset.build_interactions(train_interactions_data)
lfm_train, lfm_val = random_train_test_split(train_interactions, test_percentage=0.2)

# Filter out movies not in training set from the movies DataFrame
valid_movie_ids = set(train_df['movieId'].unique())
movies_df_filtered = movies_df[movies_df['movieId'].isin(valid_movie_ids)]

# features matrix
lfm_item_features = lfm_dataset.build_item_features(
    [(row['movieId'], row['genre_list']) for _, row in movies_df_filtered.iterrows()]
)

# filter out items not in training for the test interactions
test_interactions_data_filtered = [
    (u, i, w) for (u, i, w) in test_interactions_data 
    if i in valid_movie_ids
]
test_interactions, _ = lfm_dataset.build_interactions(test_interactions_data_filtered)


# train the LightFM model.
lightfm_model = LightFM(loss='warp', no_components=50)
lightfm_model.fit(lfm_train, item_features=lfm_item_features, epochs=30, num_threads=4)

# Evaluate 

precision = precision_at_k(lightfm_model, lfm_val, item_features=lfm_item_features, k=10).mean()
recall = recall_at_k(lightfm_model, lfm_val, item_features=lfm_item_features, k=10).mean()
auc = auc_score(lightfm_model, lfm_val, item_features=lfm_item_features).mean()

print("\nLightFM Evaluation Results:")
print(f"Precision@10: {precision:.4f}")
print(f"Recall@10: {recall:.4f}")

def compute_ndcg_at_k(model, interactions, k=10):
    n_users, n_items = interactions.shape
    ndcg_scores = []
    for user_id in range(n_users):
        true_items = interactions.tocsr()[user_id].indices
        if len(true_items) == 0:
            continue
        scores = model.predict(user_id, np.arange(n_items))
        ranked_items = np.argsort(-scores)[:k]
        dcg = sum([1.0 / np.log2(i + 2) for i, item in enumerate(ranked_items) if item in true_items])
        ideal_hits = min(len(true_items), k)
        idcg = sum([1.0 / np.log2(i + 2) for i in range(ideal_hits)])
        ndcg_scores.append(dcg / idcg if idcg > 0 else 0.0)
    return np.mean(ndcg_scores) if ndcg_scores else 0.0

ndcg_10 = compute_ndcg_at_k(lightfm_model, lfm_val, k=10)
print(f"NDCG@10: {ndcg_10:.4f}")



LightFM Evaluation Results:
Precision@10: 0.0816
Recall@10: 0.0687
NDCG@10: 0.0677


### HYBRID RECOMMENDER

In [7]:
# Instantiate the HybridRecSys model.
num_users = len(user2index)
num_movies = len(movie2index)
num_genres = genre_features.shape[1]
embedding_dim = 50

hybrid_model = HybridRecSys(num_users, num_movies, num_genres, embedding_dim)

# Set up optimizer.
optimizer_hybrid_bpr = optim.Adam(hybrid_model.parameters(), lr=0.001)
num_epochs_bpr = 10

# Train using BPR loss.
hybrid_model = hybrid_model.fit_bpr(train_loader, positive_user_items, all_movie_indices,
                                    inv_movie2index, movieid_to_index, optimizer_hybrid_bpr,
                                    num_epochs_bpr, device, genre_features)


HybridRecSys BPR Epoch 1/10: Loss 0.5508
HybridRecSys BPR Epoch 2/10: Loss 0.4028
HybridRecSys BPR Epoch 3/10: Loss 0.3516
HybridRecSys BPR Epoch 4/10: Loss 0.3191
HybridRecSys BPR Epoch 5/10: Loss 0.3025
HybridRecSys BPR Epoch 6/10: Loss 0.2868
HybridRecSys BPR Epoch 7/10: Loss 0.2759
HybridRecSys BPR Epoch 8/10: Loss 0.2679
HybridRecSys BPR Epoch 9/10: Loss 0.2597
HybridRecSys BPR Epoch 10/10: Loss 0.2527


In [8]:
evaluate_hybrid_ranking(hybrid_model, test_loader, 10, device)

Precision@10: 0.1325
Recall@10: 0.8380
NDCG@10: 0.0116


(0.13248027851651406, 0.8379853, 0.011636990117509407)

### NeuMF model 

In [9]:
# Instantiate the NeuMFHybrid model.
num_users = len(user2index)
num_movies = len(movie2index)
num_genres = genre_features.shape[1]
embedding_dim = 50
neumf_model = NeuMFHybrid(num_users, num_movies, num_genres, embedding_dim)
optimizer_neumf_bpr = optim.Adam(neumf_model.parameters(), lr=0.001)
num_epochs_bpr = 10

# Train using BPR loss.
neumf_model = neumf_model.fit_bpr(train_loader, positive_user_items, all_movie_indices,
                                   inv_movie2index, movieid_to_index, optimizer_neumf_bpr,
                                   num_epochs_bpr, device, genre_features)

NeuMFHybrid BPR Epoch 1/10: Loss 0.5467
NeuMFHybrid BPR Epoch 2/10: Loss 0.4043
NeuMFHybrid BPR Epoch 3/10: Loss 0.3494
NeuMFHybrid BPR Epoch 4/10: Loss 0.3208
NeuMFHybrid BPR Epoch 5/10: Loss 0.2994
NeuMFHybrid BPR Epoch 6/10: Loss 0.2840
NeuMFHybrid BPR Epoch 7/10: Loss 0.2719
NeuMFHybrid BPR Epoch 8/10: Loss 0.2618
NeuMFHybrid BPR Epoch 9/10: Loss 0.2535
NeuMFHybrid BPR Epoch 10/10: Loss 0.2431


In [10]:

evaluate_ranking_neumf(neumf_model, test_loader, 10, device)

Precision@10: 0.1198
Recall@10: 0.6661
NDCG@10: 0.0108


(0.11977400690403216, 0.666134, 0.0108409449004788)